<a href="https://colab.research.google.com/github/kyle-woodward/bq-ee-vectorsearch/blob/main/src/01_earthgenome_embeddings_bq_vectorsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BigQuery ELT of EarthGenome Embeddings for Vector Search

*Note: This notebook will create and consume resources on Google Cloud. Though it should be minimal, be mindful of cost and always delete resources after running demos.*

In order to run this demo you will need Google Cloud IAM permissions to:
* read, write, and create Cloud Storage objects
* read, write, and create BigQuery resources

Refer to [docs](https://cloud.google.com/iam/docs/roles-overview) for more info if you get a permissions-related error

In [ ]:
import os
import json
import geopandas as gpd
import subprocess
from google.cloud import bigquery
import datetime

### Configure AWS credentials (you'll need an AWS account and a key created)

In [ ]:
!pip install awscli
!aws --version

In [ ]:
!aws configure

### Configure Google Cloud Credentials & Resources

In [ ]:
# change to your GCS settings
BUCKET = "gs://YOUR-BUCKET" # GC Storage bucket
PROJECT_ID = "YOUR-PROJECT" # GC project
LOCATION = "YOUR-REGION" # compute region
DATASET_ID = "YOUR_DATASET" # BigQuery Dataset
TABLE_ID = "YOUR_TABLE" # BigQuery Table

In [ ]:
!gcloud auth login
!gcloud config set project {PROJECT_ID}

In [ ]:
import google.auth
scopes = ['https://www.googleapis.com/auth/cloud-platform']
creds, _ = google.auth.default(scopes=scopes, default_scopes=scopes, quota_project_id=PROJECT_ID)

## Downloading Earthgenome Geoparquet's

### Earth Genome has hosted it on Source.Coop - let's check how its organized -> [link](https://source.coop/repositories/earthgenome/earthindexembeddings/description)

### In [00_s2_tile_management.ipynb](./00_s2_tile_management.ipynb) we've already aggregated UTM tile IDs to country boundaries

#### we'll use that JSON file to help us pull only the EG parquet files we need for a country..

In [ ]:
# Read in our country-tile JSON reference
!mkdir -p ../esa_grid && wget https://raw.githubusercontent.com/kyle-woodward/bq-ee-vectorsearch/refs/heads/main/esa_grid/adm0_tiles_by_country.json -O ../esa_grid/adm0_tiles_by_country.json
tile_dict = json.load(open("../esa_grid/adm0_tiles_by_country.json"))
print(tile_dict.keys())

In [ ]:
country = "Kenya"
tiles = tile_dict[country]
tiles.sort()
print(f"{len(tiles)} S2 tiles covering {country}")
for t in tiles:
    print(t)

In [ ]:
dryrun=False

for i,t in enumerate(tiles):
    # limit data we're downloading..
    if i > 0:
        break

    suffix = "2024-01-01_2025-01-01.parquet"
    pattern = f"s3://earthgenome/earthindexembeddings/2024/{t}_{suffix}"
    cmd = f"aws s3 cp {pattern} ../embeddings/earthgenome/2024/{t}_{suffix} --endpoint-url=https://data.source.coop"
    if dryrun:
        print(cmd)
    else:
        print(f"Running {cmd}")
        try:
            subprocess.run(cmd, shell=True, capture_output=True, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Error copying {t}: {e}")
            # If the file does not exist, we can skip it
            if "does not exist" in e.stderr.decode():
                print(f"File {t} does not exist, skipping.")
                continue
            else:
                raise

Look at a geoparquet file

In [ ]:
# look at one
files = os.listdir("../embeddings/earthgenome/2024")
print(f"{len(list(files))} files:\n {files}")
file = os.path.join("../embeddings/earthgenome/2024", files[0])
print(file)
df = gpd.read_parquet(file)
print(df.head())


we'll add a tile column to help us stay organized

In [ ]:
# overwrite all files to add tile column
for file in files:
    file_path = os.path.join("../embeddings/earthgenome/2024", file)
    df = gpd.read_parquet(file_path)
    df.loc[:,'tile'] = os.path.basename(file).split("_")[0]
    df.to_parquet(file_path, index=False)
    print(f"Updated {file} with tile column.")
    break


In [ ]:
print(gpd.read_parquet(file_path).head())

### Loading Data into BigQuery

You'll need a GCS bucket and a BigQuery Dataset

In [ ]:
# create the storage bucket and BigQuery dataset
!gcloud storage buckets create {BUCKET} --location {LOCATION} --project {PROJECT_ID}
!bq mk -d --data_location={LOCATION} --project_id {PROJECT_ID} {DATASET_ID}

In [ ]:
# upload parquet files to gcs
# try gcloud storage sync..
gcloud_folder = f"{BUCKET}/earthgenome/2024"
!gcloud storage rsync ../embeddings/earthgenome/2024 $gcloud_folder \
    --project=$PROJECT_ID

In [ ]:
FULL_TABLE = f"{PROJECT_ID}:{DATASET_ID}.{TABLE_ID}"
FOLDER = "earthgenome/2024"
print(FULL_TABLE)
for i,file in enumerate(files):
    # limit what we're ingesting to BQ
    if i > 0:
        break
    URI = f"{BUCKET}/{FOLDER}/{file}"

    print(URI)
    !bq --location=$LOCATION --project_id=$PROJECT_ID \
            load \
                --source_format=PARQUET \
                $FULL_TABLE \
                $URI

### Minor transforms of the BQ table

we will do a small post-processing query on the loaded embeddings table to get the embedding field converted correctly for vector search..

vector search indexing requires the embedding field to be of type `ARRAY<FLOAT>`

the load operation turns 'embedding' field into a double-nested STRUCT data type, innermost child containing list of floats..

so we have to unpack that list from the nested structure, final data type being `ARRAY<FLOAT64>`

In [ ]:
query = f"""
SELECT
  eg.id,
  eg.tile,
  ST_GEOGFROMTEXT(grouped.geometry_text) AS geometry,
  ARRAY_AGG(e.element) AS embedding
FROM
  `{PROJECT_ID}`.`{DATASET_ID}`.`{TABLE_ID}` AS eg
CROSS JOIN
  UNNEST(eg.embedding.list) AS e
JOIN (
  SELECT id, tile, ST_ASTEXT(geometry) AS geometry_text
  FROM `{PROJECT_ID}`.`{DATASET_ID}`.`{TABLE_ID}`
  GROUP BY id, tile, geometry_text
) AS grouped ON eg.id = grouped.id AND eg.tile = grouped.tile AND ST_ASTEXT(eg.geometry) = grouped.geometry_text
GROUP BY eg.id, eg.tile, grouped.geometry_text
"""

# Run the query and save the result to a new table
result_table = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}_v1"
job_config = bigquery.QueryJobConfig(destination=result_table)
client = bigquery.Client(project=PROJECT_ID)
job = client.query(query, job_config=job_config)
job.result()  # Wait for the job to complete

In [ ]:
# Check if the result_table exists

def table_exists(client, table_id):
    try:
        client.get_table(table_id)
        print(f"Table {table_id} exists.")
        return True
    except Exception as e:
        print(f"Table {table_id} does not exist. Error: {e}")
        return False

table_exists(client, result_table)

In [ ]:
# check the resulting table's schema and data
query = f"SELECT * FROM `{result_table}` LIMIT 10"
query_job = client.query(query)
# print schema
schema = query_job.result().schema
for field in schema:
    print(f"{field.name}: {field.field_type}")
for row in query_job:
    print(row)

### Index BQ table to enable Vector Search

Following [docs](https://cloud.google.com/bigquery/docs/vector-search#create_a_vector_index) guidance

In [ ]:
# test VECTOR SEARCH operations
in_table = '.'.join(result_table.split(".")[1:])
print(f'indexing {in_table} for vector search')
query = f"""
CREATE VECTOR INDEX my_index ON {in_table}(embedding)
OPTIONS(distance_type='COSINE', index_type='IVF', ivf_options='{{"num_lists": 1000}}');
"""

# Run the query to create the index
client = bigquery.Client(project=PROJECT_ID)
job = client.query(query)
job.result()  # Wait for the job to complete

Create a test target table of 1 record to perform vector search with

In [ ]:
result_table = result_table+"_test_target"
query = f"SELECT * FROM {in_table} LIMIT 1"

job_config = bigquery.QueryJobConfig(destination=result_table)
job = client.query(query,job_config=job_config)
job.result()  # Wait for the job to complete

Run a [Vector Search](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search)!

In [ ]:
target_table = '.'.join(result_table.split(".")[1:])
print(target_table)
query = f"""
SELECT query.id AS target_id,
  query.tile AS target_tile,
  base.id AS base_id,
  base.tile AS base_tile,
  distance
FROM
  VECTOR_SEARCH(
    TABLE {in_table},
    'embedding',
    TABLE {target_table},
    top_k => 11,
    distance_type => 'COSINE',
    options => '{{"fraction_lists_to_search": 0.005}}')
ORDER BY distance
LIMIT 10
OFFSET 1;
"""

# Run the query to create the index
client = bigquery.Client(project=PROJECT_ID)
search_result_table = f"{PROJECT_ID}.{DATASET_ID}.vector_search_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
job_config = bigquery.QueryJobConfig(destination=search_result_table)
job = client.query(query,job_config=job_config)
job.result()  # Wait for the job to complete

In [ ]:
query = f"SELECT * FROM `{search_result_table}` LIMIT 10"
query_job = client.query(query)
# print schema
schema = query_job.result().schema
for field in schema:
    print(f"{field.name}: {field.field_type}")
for row in query_job:
    print(row)

### You can take a look at your newly created BQ tables and the vector search results in [BQ studio](https://console.cloud.google.com/bigquery)